# PREPARE
You don't need mongodb to run this notebook. You can just use ConversaionBufferMemmory instead. This is for testing purpose only.

To be able to run the test in this notebook, you need a `.env` file and `google_key.json` file in this folder. The `.env` file should contain the following variables:

```env
MONGODB_LOGIN = `REPLACE`
MONGODB_LOCATION = `REPLACE`

GOOGLE_APPLICATION_CREDENTIALS = google_key.json
GOOGLE_SEARCH_API = `REPLACE`
COURSE_SEARCH_ID = `REPLACE`
RANDOM_SEARCH_ID = `REPLACE`
DEGREE_SEARCH_ID = `REPLACE`
```



# Run the Test

Import module and setup the inference model

In [ ]:
import os
import sys
from dotenv import load_dotenv

load_dotenv()

from rich import print

import random
import string

#append the parent path to sys.path
sys.path.append(os.path.dirname(os.getcwd()))

from CompassUTD import CompassInference
from langchain.memory import ReadOnlySharedMemory, MongoDBChatMessageHistory, ConversationBufferMemory,ConversationBufferWindowMemory



agent = CompassInference()



### MongoDB Setup
If you want to use memmory. Just rerun the cell below to reset the memmory

In [ ]:
# Use MongoDB
connection_string = f"mongodb+srv://{str(os.getenv('MONGODB_LOGIN'))}@{str(os.getenv('MONGODB_LOCATION'))}"

random_letter_string = "Test_" + "".join([random.choice(string.ascii_letters) for _ in range(11)])

print(random_letter_string)
message_history = MongoDBChatMessageHistory(
        connection_string=connection_string, session_id= random_letter_string
    )

### Inference
Replace the `user_message` variable and rerun the cell below to see the result. The memory will be saved so you can run the cell below multiple times to see the result.

In [ ]:
user_message = """What is the contact information for ECS undergraduate advising?"""

memmory_store = ConversationBufferWindowMemory(memory_key="chat_history",chat_memory=message_history)
read_only_memory = ReadOnlySharedMemory(memory=memmory_store)

bot_message = agent.run(user_message = user_message, 
                        read_only_memory= ReadOnlySharedMemory(memory=read_only_memory)
                        )

print(bot_message)

message_history.add_user_message(user_message)
message_history.add_ai_message(bot_message)


### Delete the test data
Remove the `assert False` in the cell below and run it to delete all test data from the database.

In [ ]:
assert False
# Remove all tested chat history with the sessionID of "Test_"
import os
from dotenv import load_dotenv
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi
load_dotenv()

uri = f"mongodb+srv://{str(os.getenv('MONGODB_LOGIN'))}@{str(os.getenv('MONGODB_LOCATION'))}/?retryWrites=true&w=majority"
# Create a new client and connect to the server
client = MongoClient(uri, server_api=ServerApi(version="1"))
#get the database chat_history.message_store
db = client["chat_history"]
collection = db["message_store"]
#Find the session ID with the string "Test_"
# Define the string pattern to match in the key
string_pattern = "Test_"

# Create a regex query to match the key
query = { "SessionId": { "$regex": string_pattern } }
#Find all session ID with the string "Test_"
query_result = collection.delete_many(query)
